In [1]:
### This script will plot some graphics showing the relative locations of the ZDR and KDP maxes in the storm.
import matplotlib.pyplot as plt
import pyart
import numpy as np
import numpy.ma as ma
from metpy.units import atleast_1d, check_units, concatenate, units
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from siphon.radarserver import RadarServer
rs = RadarServer('http://thredds-aws.unidata.ucar.edu/thredds/radarServer/nexrad/level2/S3/')
#Use the IDD server link if the AWS one doesn't work, or if you're on a non-.edu domain.
#rs = RadarServer('http://thredds.ucar.edu/thredds/radarServer/nexrad/level2/IDD/')
from datetime import datetime, timedelta
from siphon.cdmr import Dataset
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from metpy.units import atleast_1d, check_units, concatenate, units
from shapely.geometry import polygon as sp
import pyproj 
import shapely.ops as ops
from shapely.ops import transform
from shapely.geometry.polygon import Polygon
from functools import partial
from shapely import geometry
import netCDF4
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage import data, img_as_float
from pyproj import Geod
from metpy.calc import get_wind_dir
import matplotlib.lines as mlines
import pandas as pd


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



C:\Users\mwilson41\AppData\Local\Continuum\Anaconda3\envs\radar\lib\site-packages\urllib3\contrib\pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [5]:
#Set angle perpendicular to FFD Z gradient
storm_relative_dir = 180
srdir = storm_relative_dir
query = rs.query()
#Here, set the initial time of the archived radar loop you want.
dt = datetime(2017,7, 11, 23, 0)
station = 'KMVX'
#Set loop duration
query.stations(station).time_range(dt, dt + timedelta(hours=.1))
cat = rs.get_catalog(query)
cat.datasets
f = 27
n = 1
storm_index = 0
scan_index = 0
g = Geod(ellps='sphere')
for item in sorted(cat.datasets.items()):
    # After looping over the list of sorted datasets, pull the actual Dataset object out
    # of our list of items and access over CDMRemote
    try:
        ds = item[1]
        radar1 = pyart.io.nexrad_cdm.read_nexrad_cdm(ds.access_urls['OPENDAP'])
        #Now let's calculate and plot specific differential phase.
        #Pre-SAILS # for below: 17
        #SAILS #: 25
        if radar1.nsweeps > 25:
            continue
        for i in range(radar1.nsweeps):
            print('in loop')
            print(radar1.nsweeps)
            radar = radar1.extract_sweeps([i])
            if ((np.mean(radar.elevation['data']) < .65) and (np.max(np.asarray(radar.fields['differential_reflectivity']['data'])) != np.min(np.asarray(radar.fields['differential_reflectivity']['data'])))):
                n = n+1
                print(np.mean(radar.elevation['data']))
                time_start = netCDF4.num2date(radar.time['data'][0], radar.time['units'])
                print(time_start)
                kdp_dict = pyart.retrieve.kdp_proc.kdp_maesaka(radar)
                print('its this line')
                radar.add_field('KDP', kdp_dict[0])
                print('heres the problem')
                # mask out last 10 gates of each ray, this removes the "ring" around the radar.
                radar.fields['differential_reflectivity']['data'][:, -10:] = np.ma.masked
                ref_ungridded = radar.fields['reflectivity']['data']
                refl_c = np.copy(ref_ungridded)
                ref_c = ma.masked_where(refl_c < 20., refl_c)
                #Get ungridded ZDR
                zdr_ungridded = radar.fields['differential_reflectivity']['data']
                zdrl_c = np.copy(zdr_ungridded)
                zdr_c = ma.masked_where(refl_c < 20, zdrl_c)

                ungrid_lons = radar.gate_longitude['data']
                ungrid_lats = radar.gate_latitude['data']
                # exclude masked gates from the gridding
                gatefilter = pyart.filters.GateFilter(radar)
                gatefilter.exclude_masked('differential_reflectivity')
                print('almost gridding')
                #Now let's grid the data
                grid = pyart.map.grid_from_radars(
                    (radar,), gatefilters=(gatefilter, ),
                    grid_shape=(1, 500, 500),
                    grid_limits=((200, 200), (-123000.0, 123000.0), (-123000.0, 123000.0)),
                    fields=['differential_reflectivity','reflectivity','KDP','cross_correlation_ratio'])
                #Get gate altitudes
                gate_altitude = radar.gate_altitude['data'][:]
                #Get the data from the grid
                ZDR = grid.fields['differential_reflectivity']['data'][0]
                REF = grid.fields['reflectivity']['data'][0]
                KDP = grid.fields['KDP']['data'][0]
                CC = grid.fields['cross_correlation_ratio']['data'][0]

                #Mask everything below 20dbz
                #import numpy.ma as ma
                ZDRmasked1 = ma.masked_where(REF < 20, ZDR)
                REFmasked = ma.masked_where(REF < 20, REF)
                #Use a 45 dBZ mask for KDP to only get areas in the storm core
                KDPmasked = ma.masked_where(REF < 50, KDP)
                KDPmasked = ma.filled(KDPmasked, fill_value = -2)
                #Filter out spots not in forward flank using gradient direction
                print('made it to smoothing')
                smoothed_ref1 = ndi.gaussian_filter(REFmasked, sigma = 2, order = 0)
                REFgradient = np.asarray(np.gradient(smoothed_ref1))
                REFgradient[0,:,:] = ma.masked_where(REF < 20, REFgradient[0,:,:])
                REFgradient[1,:,:] = ma.masked_where(REF < 20, REFgradient[1,:,:])
                print('made it through gradient')
                grad_dir1 = get_wind_dir(REFgradient[1,:,:] * units('m/s'), REFgradient[0,:,:] * units('m/s'))
                grad_dir = ma.masked_where(REF < 20, grad_dir1)
                #srdir = storm_relative_dir
                grad_ffd = np.abs(np.arctan2(np.sin(grad_dir * units('degrees')-srdir * units('degrees')), np.cos(grad_dir * units('degrees')-srdir * units('degrees'))))
                grad_ffd = grad_ffd.to('degrees')
                print('got gradient')
                #Add a fill value for the ZDR mask so that contours will be closed
                ZDRmasked2 = ma.masked_where(grad_ffd > 120 * units('degrees'), ZDRmasked1)
                ZDRmasked = ma.masked_where(CC < .60, ZDRmasked2)
                ZDRmasked = ma.filled(ZDRmasked, fill_value = -2)

                rlons = grid.point_longitude['data']
                rlats = grid.point_latitude['data']
                rlons_2d = rlons[0,:,:]
                rlats_2d = rlats[0,:,:]
                cenlat = radar.latitude['data'][0]
                cenlon = radar.longitude['data'][0]
                #Let's set up the map projection!
                print('Set up our projection')
                crs = ccrs.LambertConformal(central_longitude=-100.0, central_latitude=45.0)

                # Set up our array of latitude and longitude values and transform to 
                # the desired projection.

                tlatlons = crs.transform_points(ccrs.LambertConformal(central_longitude=265, central_latitude=25, standard_parallels=(25.,25.)),rlons[0,:,:],rlats[0,:,:])
                tlons = tlatlons[:,:,0]
                tlats = tlatlons[:,:,1]

                # Limit the extent of the map area, must convert to proper coords.
                LL = (cenlon-1.5,cenlat-1.5,ccrs.PlateCarree())
                UR = (cenlon+1.5,cenlat+1.5,ccrs.PlateCarree())
                print(LL)

                # Get data to plot state and province boundaries
                states_provinces = cfeature.NaturalEarthFeature(
                        category='cultural',
                        name='admin_1_states_provinces_lakes',
                        scale='50m',
                        facecolor='none')
                fname = 'cb_2016_us_county_20m/cb_2016_us_county_20m.shp'
                fname2 = 'cb_2016_us_state_20m/cb_2016_us_state_20m.shp'
                counties = ShapelyFeature(Reader(fname).geometries(),ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black')
                states = ShapelyFeature(Reader(fname2).geometries(),ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black')
                fig=plt.figure(n,figsize=(30.,25.))
                ax = plt.subplot(111,projection=ccrs.PlateCarree())
                ax.coastlines('50m',edgecolor='black',linewidth=0.75)
                #ax.add_feature(states_provinces,edgecolor='black',linewidth=0.5)
                ax.add_feature(counties, edgecolor = 'black', linewidth = 0.5)
                ax.add_feature(states, edgecolor = 'black', linewidth = 1.5)
                ax.set_extent([LL[0],UR[0],LL[1],UR[1]])
                REFlevels = np.arange(20,73,2)
                print('plotting')
                #ax.contourf(rlons[0,:,:],rlats[0,:,:],REFmasked,REFlevels,cmap = plt.cm.gist_ncar)
                refp = ax.pcolormesh(ungrid_lons, ungrid_lats, ref_c, cmap=plt.cm.gist_ncar, vmin = 10, vmax = 73)
                #zdrp = ax.pcolormesh(ungrid_lons, ungrid_lats, zdr_c, cmap=plt.cm.nipy_spectral, vmin = -2, vmax = 6)
                #Plot local maxes in reflectivity
                # image_max is the dilation of im with a 20*20 structuring element
                # It is used within peak_local_max function
                smoothed_ref = ndi.gaussian_filter(REFmasked, sigma = 3, order = 0)
                #image_max = ndi.maximum_filter(smoothed_ref, size=15, mode='constant')

                # Comparison between image_max and im to find the coordinates of local maxima
                #coordinates = peak_local_max(smoothed_ref, min_distance=20)
                #ref_maxes = REFmasked[coordinates[:,0], coordinates[:,1]]
                #max_lons = rlons[0,coordinates[:,0], coordinates[:,1]]
                #max_lats = rlats[0,coordinates[:,0], coordinates[:,1]]
                #max_lons_c = max_lons[ref_maxes > 45]
                #max_lats_c = max_lats[ref_maxes > 45]
                
                REFlev = [45]
                REFlev1 = [50]
                refc = ax.contour(rlons[0,:,:],rlats[0,:,:],smoothed_ref,REFlev, alpha=.01)

                ref_areas = []
                max_lons_c = []
                max_lats_c = []
                storm_ids = []
                
                proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
                           pyproj.Proj(init='epsg:3857'))

                #Look for reflectivity centroids
                for col in refc.collections:
                    # Loop through all polygons that have the same intensity level
                    for contour_path in col.get_paths(): 
                        # Create the polygon for this intensity level
                        # The first polygon in the path is the main one, the following ones are "holes"
                        for ncp,cp in enumerate(contour_path.to_polygons()):
                            print(1)
                            cpa = np.asarray(cp[:])
                            x = cpa[:,0]
                            y = cpa[:,1]
                            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
                            if ncp == 0:
                                poly = new_shape
                            else:
                                # Remove the holes if there are any
                                poly = poly.difference(new_shape)

                        # do something with polygon
                        #print(poly.area) 
                        print(poly.centroid.x)
                        s_new = transform(proj, poly)
                        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
                        boundary = np.asarray(poly.boundary.xy)
                        polypath = Path(boundary.transpose())
                        coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
                        maskr = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                        meanr = np.mean(smoothed_ref[maskr])
                        print('past mask')
                        print(meanr)
                        if projected_area > 10 * units('km^2') and meanr > REFlev[0]:
                            print('found a storm')
                            #For big blobs with embedded supercells, find the embedded storm cores
                            #Normal 'big storm' cutoff 300 km^2
                            if projected_area > 300 * units('km^2'):
                                print('found a big storm')
                                rlon_2 = rlons[0,:,:]
                                rlat_2 = rlats[0,:,:]
                                #smoothed_ref_m = ma.MaskedArray(smoothed_ref, mask=maskr)
                                smoothed_ref_m = ma.masked_where(maskr==False, smoothed_ref)
                                smoothed_ref_m = ma.filled(smoothed_ref_m, fill_value = -2)
                                rlon2m = ma.MaskedArray(rlon_2, mask=maskr)
                                rlat2m = ma.MaskedArray(rlat_2, mask=maskr)
                                refc1 = ax.contour(rlon2m,rlat2m,smoothed_ref_m,REFlev1, linewidths = 3, linestyle = '--', alpha=.01)
                                #refc1 = ax.contour(rlon_2[maskr],rlat_2[maskr],smoothed_ref[maskr],REFlev1, colors = 'g', linewidths = 3)
                                print('plotted a big storm')
                                #Look for reflectivity centroids
                                for col1 in refc1.collections:
                                    # Loop through all polygons that have the same intensity level
                                    print('made it to beginning of loop')
                                    for contour_path1 in col1.get_paths(): 
                                        # Create the polygon for this intensity level
                                        # The first polygon in the path is the main one, the following ones are "holes"
                                        for ncp1,cp1 in enumerate(contour_path1.to_polygons()):
                                            print(2)
                                            cpa1 = np.asarray(cp1[:])
                                            x1 = cpa1[:,0]
                                            y1 = cpa1[:,1]
                                            new_shape1 = geometry.Polygon([(i[0], i[1]) for i in zip(x1,y1)])
                                            if ncp1 == 0:
                                                poly1 = new_shape1
                                            else:
                                                # Remove the holes if there are any
                                                poly1 = poly1.difference(new_shape)

                                        # do something with polygon
                                        #print(poly.area) 
                                        print(poly1.centroid.x)
                                        s_new1 = transform(proj, poly1)
                                        projected_area1 = (transform(proj, poly1).area * units('m^2')).to('km^2')
                                        if projected_area1 > 10 * units('km^2'):
                                            ref_areas.append((projected_area1*2))
                                            max_lons_c.append((poly1.centroid.x))
                                            max_lats_c.append((poly1.centroid.y))
                                            if scan_index == 0:
                                                storm_ids.append((storm_index))
                                                storm_index = storm_index + 1
                                            else:
                                                #dist_track = np.zeros((np.asarray(max_lons_p).shape[0]))
                                                max_lons_p = np.asarray(tracks_dataframe['storm_lon'].loc[scan_index-1].iloc[:])
                                                max_lats_p = np.asarray(tracks_dataframe['storm_lat'].loc[scan_index-1].iloc[:])
                                                storm_ids_p = np.asarray(tracks_dataframe['storm_id1'].loc[scan_index-1].iloc[:])
                                                dist_track = np.zeros((np.asarray(max_lons_p).shape[0]))
                                                for i in range(max_lons_p.shape[0]):
                                                    distance_track = g.inv(poly1.centroid.x, poly1.centroid.y,
                                                                           max_lons_p[i], max_lats_p[i])
                                                    dist_track[i] = distance_track[2]/1000.
                                                print(dist_track)
                                                print('Poly lon', poly1.centroid.x)
                                                print(max_lons_p)
                                                print(storm_ids_p)
                                                if np.min(dist_track) < 10.0:
                                                    storm_ids.append((storm_ids_p[np.where(dist_track == np.min(dist_track))[0][0]]))
                                                    print('storm id', storm_ids_p[np.where(dist_track == np.min(dist_track))[0][0]])
                                                else:
                                                    storm_ids.append((storm_index))
                                                    print('storm id', storm_ids_p[np.where(dist_track == np.min(dist_track))[0][0]])
                                                    storm_index = storm_index + 1
                                            print('added polygon')
                                        else:
                                            print('nope')
                            else:
                                ref_areas.append((projected_area))
                                max_lons_c.append((poly.centroid.x))
                                max_lats_c.append((poly.centroid.y))
                                if scan_index == 0:
                                    storm_ids.append((storm_index))
                                    storm_index = storm_index + 1
                                else:
                                    #dist_track = np.zeros((np.asarray(max_lons_p).shape[0]))
                                    max_lons_p = np.asarray(tracks_dataframe['storm_lon'].loc[scan_index-1].iloc[:])
                                    max_lats_p = np.asarray(tracks_dataframe['storm_lat'].loc[scan_index-1].iloc[:])
                                    storm_ids_p = np.asarray(tracks_dataframe['storm_id1'].loc[scan_index-1].iloc[:])
                                    dist_track = np.zeros((np.asarray(max_lons_p).shape[0]))
                                    for i in range(max_lons_p.shape[0]):
                                        distance_track = g.inv(poly.centroid.x, poly.centroid.y,
                                                               max_lons_p[i], max_lats_p[i])
                                        dist_track[i] = distance_track[2]/1000.
                                    print(dist_track)
                                    print('Poly lon', poly.centroid.x)
                                    print(max_lons_p)
                                    print(storm_ids_p)
                                    if np.min(dist_track) < 10.0:
                                        storm_ids.append((storm_ids_p[np.where(dist_track == np.min(dist_track))[0][0]]))
                                        print('storm id', storm_ids_p[np.where(dist_track == np.min(dist_track))[0][0]])
                                    else:
                                        storm_ids.append((storm_index))
                                        print('storm id', storm_ids_p[np.where(dist_track == np.min(dist_track))[0][0]])
                                        storm_index = storm_index + 1
                                print('added polygon')
                                            
                    #print(s_new)
                max_lons_c = np.asarray(max_lons_c)
                max_lats_c = np.asarray(max_lats_c)
                #ZDRlevels = np.arange(3, np.max(ZDRmasked)+((np.max(ZDRmasked))-3)/2, (np.max(ZDRmasked))-3)
                zdrlev = [3.5]
                kdplev = [.75]
                #ZDRlevels = np.arange(3,5.5,.5)
                #ZDRlevels1 = np.arange(5,10,.5)
                #KDPlevels = np.arange(.75, np.max(KDPmasked)+((np.max(KDPmasked))-1.5)/2, (np.max(KDPmasked))-1.5)
                #KDPlevels = np.arange(.75,1.75,.25)
                #KDPlevels1 = np.arange(1.5,10,.25)
                zdrc = ax.contour(rlons[0,:,:],rlats[0,:,:],ZDRmasked,zdrlev,linewidths = 2, colors='purple', alpha = .01)
                #zrdc = ax.contourf(rlons[0,:,:],rlats[0,:,:],ZDRmasked,ZDRlevels,linewide = .01, colors='pink', alpha = .8)
                #ax.contourf(rlons[0,:,:],rlats[0,:,:],ZDRmasked,ZDRlevels1,linewide = .01, colors='crimson', alpha = .8)

                #kdpc = ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels,linewide = .01, colors ='green', alpha = .5)
                kdpc = ax.contour(rlons[0,:,:],rlats[0,:,:],KDPmasked,kdplev,linewidths = 2, colors='green', alpha = .8)
                #ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels1,linewide = .01, colors ='b', alpha = .5)
                print('made it here')
                plt.savefig('testfig.png')

                #proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
                #           pyproj.Proj(init='epsg:3857'))
                zdr_areas = []
                zdr_centroid_lon = []
                zdr_centroid_lat = []
                zdr_mean = []
                zdr_cc_mean = []
                zdr_max = []
                zdr_storm_lon = []
                zdr_storm_lat = []
                zdr_dist = []
                zdr_forw = []
                zdr_back = []
                #print("here too")
                for col in zdrc.collections:
                    # Loop through all polygons that have the same intensity level
                    #print('hi')
                    for contour_path in col.get_paths(): 
                        # Create the polygon for this intensity level
                        # The first polygon in the path is the main one, the following ones are "holes"
                        #print('hi')
                        for ncp,cp in enumerate(contour_path.to_polygons()):
                            #print('hi')
                            cpa = np.asarray(cp[:])
                            x = cpa[:,0]
                            y = cpa[:,1]
                            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
                            if ncp == 0:
                                poly = new_shape
                                #print('hi')
                            else:
                                # Remove the holes if there are any
                                poly = poly.difference(new_shape)
                                #print('hi')

                        # do something with polygon
                        #print(poly.area) 
                        #print(poly.centroid.x)
                        s_new = transform(proj, poly)
                        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
                        boundary = np.asarray(poly.boundary.xy)
                        polypath = Path(boundary.transpose())
                        coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
                        mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                        mean = np.mean(ZDRmasked[mask])
                        mean_cc = np.mean(CC[mask])
                        if projected_area > 1 * units('km^2') and mean > zdrlev[0] and mean_cc > .88:
                            g = Geod(ellps='sphere')
                            dist = np.zeros((np.asarray(max_lons_c).shape[0]))
                            forw = np.zeros((np.asarray(max_lons_c).shape[0]))
                            back = np.zeros((np.asarray(max_lons_c).shape[0]))
                            for i in range(dist.shape[0]):
                                        distance_1 = g.inv(poly.centroid.x, poly.centroid.y,
                                                               max_lons_c[i], max_lats_c[i])
                                        #print(distance_1[2]/1000)
                                        #print(distance_1)
                                        back[i] = distance_1[1]
                                        if distance_1[1] < 0:
                                            back[i] = distance_1[1] + 360
                                        forw[i] = np.abs(back[i] - storm_relative_dir)
                                        dist[i] = distance_1[2]/1000.
                            #print(dist.shape)
                            if (forw[np.where(dist == np.min(dist))[0][0]] < 90 and np.min(dist) < 20.0) or (forw[np.where(dist == np.min(dist))[0][0]] < 140 and np.min(dist) < 6.0):
                                zdr_storm_lon.append((max_lons_c[np.where(dist == np.min(dist))[0][0]]))
                                zdr_storm_lat.append((max_lats_c[np.where(dist == np.min(dist))[0][0]]))
                                zdr_dist.append(np.min(dist))
                                zdr_forw.append(forw[np.where(dist == np.min(dist))[0][0]])
                                zdr_back.append(back[np.where(dist == np.min(dist))[0][0]])
                                zdr_areas.append((projected_area))
                                zdr_centroid_lon.append((poly.centroid.x))
                                zdr_centroid_lat.append((poly.centroid.y))
                                zdr_mean.append((mean))
                                zdr_cc_mean.append((mean_cc))
                                zdr_max.append((np.max(ZDRmasked[mask])))
                                patch = PathPatch(polypath, facecolor='none', alpha=.8, edgecolor = 'blue', linewidth = 3)
                                ax.add_patch(patch)
                        #print(s_new)
                print('made it through zdr centroids')
                
                if len(zdr_storm_lon) > 0:
                    kdp_areas = []
                    kdp_centroid_lon = []
                    kdp_centroid_lat = []
                    kdp_max = []
                    kdp_storm_lon = []
                    kdp_storm_lat = []
                    for col in kdpc.collections:
                        # Loop through all polygons that have the same intensity level
                        for contour_path in col.get_paths(): 
                            # Create the polygon for this intensity level
                            # The first polygon in the path is the main one, the following ones are "holes"
                            for ncp,cp in enumerate(contour_path.to_polygons()):
                                print(1)
                                cpa = np.asarray(cp[:])
                                x = cpa[:,0]
                                y = cpa[:,1]
                                new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
                                if ncp == 0:
                                    poly = new_shape
                                else:
                                    # Remove the holes if there are any
                                    poly = poly.difference(new_shape)

                            # do something with polygon
                            #print(poly.area) 
                            print(poly.centroid.x)
                            s_new = transform(proj, poly)
                            projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
                            boundary = np.asarray(poly.boundary.xy)
                            polypath = Path(boundary.transpose())
                            coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
                            mask_kdp = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                            #mean = np.mean(ZDRmasked[mask])
                            #mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                            #mean = np.mean(REFmasked[mask])
                            if projected_area > 2 * units('km^2'):
                                g = Geod(ellps='sphere')
                                dist_kdp = np.zeros((np.asarray(max_lons_c).shape[0]))
                                for i in range(dist_kdp.shape[0]):
                                            distance_kdp = g.inv(poly.centroid.x, poly.centroid.y,
                                                                   max_lons_c[i], max_lats_c[i])
                                            #print(distance_1[2]/1000)
                                            #print("KDP dist:", distance_kdp)
                                            dist_kdp[i] = distance_kdp[2]/1000.
                                print(dist_kdp)
                                if np.min(np.asarray(dist_kdp)) < 15.0:
                                    #print('Got to KDP stuff')
                                    kdp_areas.append((projected_area))
                                    kdp_centroid_lon.append((poly.centroid.x))
                                    kdp_centroid_lat.append((poly.centroid.y))
                                    kdp_storm_lon.append((max_lons_c[np.where(dist_kdp == np.min(dist_kdp))[0][0]]))
                                    kdp_storm_lat.append((max_lats_c[np.where(dist_kdp == np.min(dist_kdp))[0][0]]))
                                    kdp_max.append((np.max(KDPmasked[mask_kdp])))
                                    patch = PathPatch(polypath, facecolor='none', alpha=.5, edgecolor = 'grey', linewidth = 3)
                                    ax.add_patch(patch)
                
                    print('made it through kdp centroids')

                    #Consolidating the arc objects associated with each storm:
                    zdr_areas_arr = np.zeros((len(zdr_areas)))
                    zdr_max_arr = np.zeros((len(zdr_max)))
                    zdr_mean_arr = np.zeros((len(zdr_mean)))                    
                    for i in range(len(zdr_areas)):
                        zdr_areas_arr[i] = zdr_areas[i].magnitude
                        zdr_max_arr[i] = zdr_max[i]
                        zdr_mean_arr[i] = zdr_mean[i]

                    zdr_centroid_lons = np.asarray(zdr_centroid_lon)
                    zdr_centroid_lats = np.asarray(zdr_centroid_lat)
                    zdr_con_areas = []
                    zdr_con_maxes = []
                    zdr_con_means = []
                    zdr_con_centroid_lon = []
                    zdr_con_centroid_lat = []
                    zdr_con_max_lon = []
                    zdr_con_max_lat = []
                    zdr_con_storm_lon = []
                    zdr_con_storm_lat = []

                    #For KDP as well
                    kdp_areas_arr = np.zeros((len(kdp_areas)))
                    kdp_max_arr = np.zeros((len(kdp_max)))
                    for i in range(len(kdp_areas)):
                        kdp_areas_arr[i] = kdp_areas[i].magnitude
                        kdp_max_arr[i] = kdp_max[i]
                    kdp_centroid_lons = np.asarray(kdp_centroid_lon)
                    kdp_centroid_lats = np.asarray(kdp_centroid_lat)
                    kdp_con_areas = []
                    kdp_con_maxes = []
                    kdp_con_centroid_lon = []
                    kdp_con_centroid_lat = []
                    kdp_con_max_lon = []
                    kdp_con_max_lat = []
                    kdp_con_storm_lon = []
                    kdp_con_storm_lat = []
                    for i in enumerate(zdr_storm_lon):
                        print(i[0])
                        if i[0] != 0:
                            if zdr_storm_lon[i[0]-1] == zdr_storm_lon[i[0]]:
                                #print("Skipping this one")
                                continue
                            else:
                                print(zdr_storm_lon[i[0]])
                                #Find the arc objects associated with this storm:
                                zdr_objects_lons = zdr_centroid_lons[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                zdr_objects_lats = zdr_centroid_lats[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                #print("zdr lons:", zdr_objects_lons)
                                #Get the sum of their areas
                                print(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
                                zdr_con_areas.append(np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                zdr_con_maxes.append(np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                zdr_con_means.append(np.mean(zdr_mean_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                zdr_con_max_lon.append(rlons_2d[np.where(ZDRmasked==np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))])
                                zdr_con_max_lat.append(rlats_2d[np.where(ZDRmasked==np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))])
                                #print("Areas sum:", zdr_con_areas)
                                #Find the actual centroids
                                weighted_lons = zdr_objects_lons * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                zdr_con_centroid_lon.append(np.sum(weighted_lons) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                weighted_lats = zdr_objects_lats * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                                zdr_con_centroid_lat.append(np.sum(weighted_lats) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                                zdr_con_storm_lon.append(zdr_storm_lon[i[0]])
                                zdr_con_storm_lat.append(zdr_storm_lat[i[0]])
                                #print("New centroid lon:", zdr_con_centroid_lon, "New centroid lat:", zdr_con_centroid_lat)
                                #print("lons in loop", zdr_objects_lons)

                                try:
                                    #Find the kdp objects associated with this storm:
                                    kdp_objects_lons = kdp_centroid_lons[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                    #print("kdp lons:", kdp_objects_lons)
                                    kdp_objects_lats = kdp_centroid_lats[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                    #Get the sum of their areas
                                    print(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                                    kdp_con_areas.append(np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                    kdp_con_maxes.append(np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                    kdp_con_max_lon.append(rlons_2d[np.where(KDPmasked==np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))])
                                    kdp_con_max_lat.append(rlats_2d[np.where(KDPmasked==np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))])
                                    #Find the actual centroids
                                    weighted_lons_kdp = kdp_objects_lons * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                    kdp_con_centroid_lon.append(np.sum(weighted_lons_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                    weighted_lats_kdp = kdp_objects_lats * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                    #print("Could be it:","weighted lons:",weighted_lons_kdp, "object lons",kdp_objects_lons, "areas:",kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                                    kdp_con_centroid_lat.append(np.sum(weighted_lats_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                    kdp_con_storm_lon.append(zdr_storm_lon[i[0]])
                                    kdp_con_storm_lat.append(zdr_storm_lat[i[0]])
                                except:
                                    print('storm missing kdp or zdr')
                                    kdp_con_areas.append(0)
                                    kdp_con_maxes.append(0)
                                    kdp_con_max_lon.append(0)
                                    kdp_con_max_lat.append(0)
                                    kdp_con_centroid_lon.append(0)
                                    kdp_con_centroid_lat.append(0)
                                    kdp_con_storm_lon.append(0)
                                    kdp_con_storm_lat.append(0)



                        else:
                            #print(zdr_storm_lon[i[0]])
                            #Find the arc objects associated with this storm:
                            zdr_objects_lons = zdr_centroid_lons[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                            zdr_objects_lats = zdr_centroid_lats[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                            #print("zdr lons:", zdr_objects_lons)
                            #print("arc lats:", zdr_objects_lats)
                            #Get the sum of their areas
                            #print(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])])
                            zdr_con_areas.append(np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                            zdr_con_maxes.append(np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                            zdr_con_means.append(np.mean(zdr_mean_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                            zdr_con_max_lon.append(rlons_2d[np.where(ZDRmasked==np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))])
                            zdr_con_max_lat.append(rlats_2d[np.where(ZDRmasked==np.max(zdr_max_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))])
                            #print("Areas sum:",zdr_con_areas)
                            #Find the actual centroids
                            weighted_lons = zdr_objects_lons * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                            zdr_con_centroid_lon.append(np.sum(weighted_lons) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                            weighted_lats = zdr_objects_lats * zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]
                            zdr_con_centroid_lat.append(np.sum(weighted_lats) / np.sum(zdr_areas_arr[np.where(zdr_storm_lon == zdr_storm_lon[i[0]])]))
                            zdr_con_storm_lon.append(zdr_storm_lon[i[0]])
                            zdr_con_storm_lat.append(zdr_storm_lat[i[0]])
                            #print("New centroid lon:", zdr_con_centroid_lon, "New centroid lat:", zdr_con_centroid_lat)
                            #print("lons out of loop", zdr_objects_lons)
                            try:
                                #Find the kdp objects associated with this storm:
                                kdp_objects_lons = kdp_centroid_lons[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                #print("kdp lons:", kdp_objects_lons)
                                kdp_objects_lats = kdp_centroid_lats[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                #Get the sum of their areas
                                #print(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                                kdp_con_areas.append(np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                kdp_con_maxes.append(np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                kdp_con_max_lon.append(rlons_2d[np.where(KDPmasked==np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))])
                                kdp_con_max_lat.append(rlats_2d[np.where(KDPmasked==np.max(kdp_max_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))])
                                #Find the actual centroids
                                weighted_lons_kdp = kdp_objects_lons * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                kdp_con_centroid_lon.append(np.sum(weighted_lons_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                weighted_lats_kdp = kdp_objects_lats * kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]
                                #print("Could be it:","weighted lons:",weighted_lons_kdp, "object lons",kdp_objects_lons, "areas:",kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])])
                                kdp_con_centroid_lat.append(np.sum(weighted_lats_kdp) / np.sum(kdp_areas_arr[np.where(kdp_storm_lon == zdr_storm_lon[i[0]])]))
                                kdp_con_storm_lon.append(zdr_storm_lon[i[0]])
                                kdp_con_storm_lat.append(zdr_storm_lat[i[0]])
                            except:
                                print('storm missing kdp or zdr')
                                kdp_con_areas.append(0)
                                kdp_con_maxes.append(0)
                                kdp_con_max_lon.append(0)
                                kdp_con_max_lat.append(0)
                                kdp_con_centroid_lon.append(0)
                                kdp_con_centroid_lat.append(0)
                                kdp_con_storm_lon.append(0)
                                kdp_con_storm_lat.append(0)
                                    
                    #Calculate KDP-ZDR separation
                    print('calculating separation')
                    kdp_con_centroid_lons1 = np.asarray(kdp_con_centroid_lon)
                    kdp_con_centroid_lats1 = np.asarray(kdp_con_centroid_lat)
                    zdr_con_centroid_lons1 = np.asarray(zdr_con_centroid_lon)
                    zdr_con_centroid_lats1 = np.asarray(zdr_con_centroid_lat)
                    #Eliminate consolidated arcs smaller than a specified area
                    area = 2
                    zdr_con_areas_arr = np.asarray(zdr_con_areas)
                    zdr_con_centroid_lats = zdr_con_centroid_lats1[zdr_con_areas_arr > area]
                    zdr_con_centroid_lons = zdr_con_centroid_lons1[zdr_con_areas_arr > area]
                    kdp_con_centroid_lats = kdp_con_centroid_lats1[zdr_con_areas_arr > area]
                    kdp_con_centroid_lons = kdp_con_centroid_lons1[zdr_con_areas_arr > area]
                    zdr_con_max_lons1 = np.asarray(zdr_con_max_lon)[zdr_con_areas_arr > area]
                    zdr_con_max_lats1 = np.asarray(zdr_con_max_lat)[zdr_con_areas_arr > area]
                    kdp_con_max_lons1 = np.asarray(kdp_con_max_lon)[zdr_con_areas_arr > area]
                    kdp_con_max_lats1 = np.asarray(kdp_con_max_lat)[zdr_con_areas_arr > area]
                    print('Boolean problem here')
                    zdr_con_areas1 = zdr_con_areas_arr[zdr_con_areas_arr > area]

                    kdp_inds = np.where(kdp_con_centroid_lats > 0)
                    distance_kdp_zdr = g.inv(kdp_con_centroid_lons[kdp_inds], kdp_con_centroid_lats[kdp_inds], zdr_con_centroid_lons[kdp_inds], zdr_con_centroid_lats[kdp_inds])
                    dist_kdp_zdr = distance_kdp_zdr[2] / 1000.
                    #Now make an array for the distances which will have the same shape as the lats to prevent errors
                    shaped_dist = np.zeros((np.shape(zdr_con_areas)))
                    shaped_dist[kdp_inds] = dist_kdp_zdr
                    print('maybe its here')
                    #Do the same for the distances between the maxes
                    distance_kdp_zdr_max = g.inv(kdp_con_max_lons1[kdp_inds], kdp_con_max_lats1[kdp_inds], zdr_con_max_lons1[kdp_inds], zdr_con_max_lats1[kdp_inds])
                    dist_kdp_zdr_max = distance_kdp_zdr_max[2] / 1000.
                    #Now make an array for the distances which will have the same shape as the lats to prevent errors
                    shaped_dist_max = np.zeros((np.shape(zdr_con_areas)))
                    shaped_dist_max[kdp_inds] = dist_kdp_zdr_max
                    print('or not')
                    
                else:
                    print('No ZDR arcs')
                    kdp_areas = []
                    kdp_centroid_lon = []
                    kdp_centroid_lat = []
                    kdp_storm_lon = []
                    kdp_storm_lat = []
                    zdr_con_centroid_lats = []
                    zdr_con_centroid_lons = []
                    kdp_con_centroid_lats = []
                    kdp_con_centroid_lons = []
                    kdp_con_area = []
                    zdr_con_areas1 = []
                    
                ###Now let's consolidate everything to fit the Pandas dataframe!
                p_zdr_areas = []
                p_zdr_maxes = []
                p_zdr_means = []
                p_separations = []
                if len(zdr_storm_lon) > 0:
                    for storm in enumerate(max_lons_c):
                        print(storm)
                        print(np.flatnonzero(np.isclose(max_lons_c[storm[0]], zdr_con_storm_lon, rtol=1e-05)))
                        matching_ind = np.flatnonzero(np.isclose(max_lons_c[storm[0]], zdr_con_storm_lon, rtol=1e-05))
                        if matching_ind.shape[0] > 0:
                            p_zdr_areas.append((zdr_con_areas[matching_ind[0]]))
                            p_zdr_maxes.append((zdr_con_maxes[matching_ind[0]]))
                            p_zdr_means.append((zdr_con_means[matching_ind[0]]))
                            p_separations.append((shaped_dist[matching_ind[0]]))
                        else:
                            p_zdr_areas.append((0))
                            p_zdr_maxes.append((0))
                            p_zdr_means.append((0))
                            p_separations.append((0))
                else:
                    for storm in enumerate(max_lons_c):
                        p_zdr_areas.append((0))
                        p_zdr_maxes.append((0))
                        p_zdr_means.append((0))
                        p_separations.append((0))
                    
                #Now start plotting stuff!
                print('made it through giant if statement')
                if np.asarray(zdr_centroid_lon).shape[0] > 0:
                    ax.scatter(zdr_centroid_lon, zdr_centroid_lat, marker = '*', s = 100, color = 'black', zorder = 10, transform=ccrs.PlateCarree())
                    #ax.scatter(zdr_con_max_lon, zdr_con_max_lat, marker = '*', s = 100, color = 'purple', zorder = 10, transform=ccrs.PlateCarree())
                if np.asarray(kdp_centroid_lon).shape[0] > 0:
                    ax.scatter(kdp_centroid_lon, kdp_centroid_lat, marker = '^', s = 100, color = 'black', zorder = 10, transform=ccrs.PlateCarree())
                    #ax.scatter(kdp_con_max_lon, kdp_con_max_lat, marker = '^', s = 100, color = 'purple', zorder = 10, transform=ccrs.PlateCarree())
                print("plotted centroids")
                #Uncomment to print all object areas
                #for i in enumerate(zdr_areas):
                #    plt.text(zdr_centroid_lon[i[0]]+.016, zdr_centroid_lat[i[0]]+.016, "%.2f km^2" %(zdr_areas[i[0]].magnitude), size = 23)
                    #plt.text(zdr_centroid_lon[i[0]]+.016, zdr_centroid_lat[i[0]]+.016, "%.2f km^2 / %.2f km / %.2f dB" %(zdr_areas[i[0]].magnitude, zdr_dist[i[0]], zdr_forw[i[0]]), size = 23)
                    #plt.annotate(zdr_areas[i[0]], (zdr_centroid_lon[i[0]],zdr_centroid_lat[i[0]]))
                #ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels1,linewide = .01, colors ='b', alpha = .5)
                #plt.tight_layout()
                #plt.savefig('ZDRarcannotated.png')
                storm_times = []
                for l in range(len(max_lons_c)):
                    storm_times.append((time_start))
                #Now record all data in a Pandas dataframe, using similar code as Tint (kind of borrowed from it)
                print('making dataframe')
                new_cells = pd.DataFrame({
                    'scan': scan_index,
                    'storm_id' : storm_ids,
                    'storm_id1' : storm_ids,
                    'storm_lon' : max_lons_c,
                    'storm_lat' : max_lats_c,
                    'zdr_area' : p_zdr_areas,
                    'zdr_max' : p_zdr_maxes,
                    'zdr_mean' : p_zdr_means,
                    'kdp_zdr_sep' : p_separations,
                    'times' : storm_times
                })
                print('setting index')
                new_cells.set_index(['scan', 'storm_id'], inplace=True)
                if scan_index == 0:
                    print('first dataframe')
                    tracks_dataframe = new_cells
                else:
                    tracks_dataframe = tracks_dataframe.append(new_cells)
                n = n+1
                scan_index = scan_index + 1
                #max_lons_p = max_lons_c
                #max_lats_p = max_lats_c
                #storm_ids_p = storm_ids
                #Plot the consolidated stuff!
                print('made it to plotting')
                if len(zdr_con_areas1) > 0:
                    #ax.scatter(zdr_con_centroid_lon, zdr_con_centroid_lat, marker = '*', s = 500, color = 'orange', zorder = 10, transform=ccrs.PlateCarree())
                    try:
                        for i in enumerate(zdr_con_centroid_lats):
                            print("consolidated ZDR:")
                            ax.scatter(zdr_con_centroid_lons, zdr_con_centroid_lats, marker = '*', s = 500, color = 'orange', zorder = 10, transform=ccrs.PlateCarree())
                            try:
                                plt.text(zdr_con_centroid_lons[i[0]]+.025, zdr_con_centroid_lats[i[0]]+.016, "%.2f km^2 / %.2f dB" %(zdr_con_areas1[i[0]], zdr_con_maxes[i[0]]), size = 23)
                            except:
                                print("oops zdr")
                        #plt.text(kdp_con_centroid_lon[i[0]]-.20, kdp_con_centroid_lat[i[0]]+.016, "%.2f km" %(dist_kdp_zdr[i[0]]), size = 23, color = 'red')                
                    except:
                        print('failed')
                        try:
                            plt.text(float(zdr_con_centroid_lons)+.016, float(zdr_con_centroid_lats)+.016, "%.2f km^2 / %.2f dB" %(zdr_con_areas1[i[0]], zdr_con_maxes[i[0]]), size = 23)
                        except:
                            print('no zdr centroids')
                        #plt.text(float(kdp_con_centroid_lon)-.20, float(kdp_con_centroid_lat)+.016, "%.2f km" %(float(dist_kdp_zdr[0])), size = 23, color = 'red')
                    if len(kdp_con_areas) > 0:
                        #ax.scatter(zdr_con_centroid_lon, zdr_con_centroid_lat, marker = '*', s = 500, color = 'orange', zorder = 10, transform=ccrs.PlateCarree())
                        try:
                            for i in kdp_inds[0]:
                                #plt.text(zdr_con_centroid_lon[i[0]]+.025, zdr_con_centroid_lat[i[0]]+.016, "%.2f km^2" %(zdr_con_areas[i[0]].magnitude), size = 23)
                                try:
                                    plt.text(kdp_con_centroid_lons[i]-.20, kdp_con_centroid_lats[i]+.016, "%.2f km" %(shaped_dist[i]), size = 23, color = 'red')                
                                except:
                                    print('oops kdp')
                        except:
                            print('failed')
                            #plt.text(float(zdr_con_centroid_lon)+.016, float(zdr_con_centroid_lat)+.016, "%.2f km^2" %(float(zdr_con_areas[0])), size = 23)
                            try:
                                plt.text(float(kdp_con_centroid_lons)-.20, float(kdp_con_centroid_lats)+.016, "%.2f km" %(float(shaped_dist[0])), size = 23, color = 'red')
                            except:
                                print('no kdp centroids')
                    else:
                        print('No kdp')
                else:
                    print('No zdr arcs')
                print("means there's a kdp problem")
                hour = time_start.hour
                if hour < 10:
                    hour = '0'+str(hour)
                minute = time_start.minute
                if minute < 10:
                    minute = '0'+str(minute)
                day = time_start.day
                if day < 10:
                    day = '0'+str(day)
                title_plot = plt.title(station+' Radar Reflectivity, ZDR, and KDP '+str(time_start.year)+'-'+str(time_start.month)+'-'+str(time_start.day)+
                                           ' '+str(hour)+':'+str(minute)+' UTC', size = 25)
                #if np.asarray(zdr_storm_lon).shape[0] > 0:
                #    ax.scatter(zdr_storm_lon,zdr_storm_lat, marker = "o", color = 'purple', s = 500)
                #if np.asarray(kdp_storm_lon).shape[0] > 0:
                #    ax.scatter(kdp_storm_lon,kdp_storm_lat, marker = "o", color = 'purple', s = 500)
                try:
                    ax.scatter(max_lons_c,max_lats_c, marker = "o", color = 'k', s = 500, alpha = .6)
                except:
                    "No storm centroids found"
                try:
                    plt.plot([zdr_con_centroid_lons[kdp_inds], kdp_con_centroid_lons[kdp_inds]], [zdr_con_centroid_lats[kdp_inds],kdp_con_centroid_lats[kdp_inds]], color = 'k', linewidth = 5, transform=ccrs.PlateCarree())
                except:
                    print('KDP-ZDR separation didt work')
                ref_centroid_lon = max_lons_c
                ref_centroid_lat = max_lats_c
                for i in enumerate(ref_centroid_lon): 
                    plt.text(ref_centroid_lon[i[0]]+.016, ref_centroid_lat[i[0]]+.016, "storm_id: %.1f" %(storm_ids[i[0]]), size = 25)
                #Comment out this line if not plotting tornado tracks
                #plt.plot([start_torlons, end_torlons], [start_torlats, end_torlats], color = 'purple', linewidth = 5, transform=ccrs.PlateCarree())
                #Add legend stuff
                zdr_outline = mlines.Line2D([], [], color='blue', linewidth = 5, linestyle = 'solid', label='ZDR Arc Outline(Area/Max)')
                kdp_outline = mlines.Line2D([], [], color='green', linewidth = 5,linestyle = 'solid', label='"KDP Foot" Outline')
                separation_vector = mlines.Line2D([], [], color='black', linewidth = 5,linestyle = 'solid', label='KDP/ZDR Centroid Separation Vector (Red Text=Distance)')
                tor_track = mlines.Line2D([], [], color='purple', linewidth = 5,linestyle = 'solid', label='Tornado Tracks')
                elevation = mlines.Line2D([], [], color='grey', linewidth = 5,linestyle = 'solid', label='Height AGL (m)')

                plt.legend(handles=[zdr_outline, kdp_outline, separation_vector, tor_track, elevation], loc = 3, fontsize = 25)
                alt_levs = [1000, 2000]
                cele = ax.contour(ungrid_lons,ungrid_lats,gate_altitude,alt_levs, linewidths = 7, alpha = .6, colors = 'grey')
                plt.clabel(cele, fontsize=18, inline=1, inline_spacing=10, fmt='%i', rightside_up=True, use_clabeltext=True)
                plt.tight_layout()
                plt.savefig('Example_ZDRArc_comp'+station+str(time_start.year)+str(time_start.month)+str(day)+str(hour)+str(minute)+'.png')
                print('figure saved')
                plt.close()

    except:
        continue

plt.show()

in loop
21
0.44664
2017-07-11 23:00:03.084000
its this line
heres the problem
almost gridding
made it to smoothing
made it through gradient
got gradient
Set up our projection
(-98.825555555555553, 46.027777777777779, <cartopy.crs.PlateCarree object at 0x000002050AFA79E8>)
plotting
1
-97.1934992584136
past mask
47.3734
found a storm
added polygon
1
-97.69793959135923
past mask
48.1268
found a storm
added polygon
1
-97.42070663509443
past mask
48.2905
found a storm
found a big storm
plotted a big storm
made it to beginning of loop
2
-97.45426796840373
added polygon
2
-97.60889622016505
nope
2
-97.61506305349813
nope
2
-97.35324553835726
added polygon
made it here
made it through zdr centroids
1
-97.25847127071187
[  4.77580462  81.18627664  47.02455746  30.94296107]
1
-97.11359553646513
1
-97.74011319832384
[ 83.78479127   3.29831057  35.6457257   51.89535964]
1
-97.58810426781373
1
-97.46283865335688
[ 48.64909764  34.69814364   0.72146509  16.40688364]
1
-97.62221562993776
[ 54.3900230

C:\Users\mwilson41\AppData\Local\Continuum\Anaconda3\envs\radar\lib\site-packages\ipykernel_launcher.py:641: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


figure saved
in loop
21
in loop
21
in loop
21
in loop
21
in loop
21
in loop
21
0.445614
2017-07-11 23:01:54.062000
its this line
heres the problem
almost gridding
made it to smoothing
made it through gradient
got gradient
Set up our projection
(-98.825555555555553, 46.027777777777779, <cartopy.crs.PlateCarree object at 0x00000205005C1728>)
plotting
1
-98.32695967470418
past mask
nan
1
-97.2223701674984
past mask
47.1039
found a storm
[  2.12243266  82.25971847  47.87642521  31.61536127]
Poly lon -97.2223701674984
[-97.19349926 -97.69793959 -97.45426797 -97.35324554]
[0 1 2 3]
storm id 0
added polygon
1
-97.10865280683211
past mask
45.6042
found a storm
[  6.28775837  86.91559199  52.20328127  35.81146171]
Poly lon -97.10865280683211
[-97.19349926 -97.69793959 -97.45426797 -97.35324554]
[0 1 2 3]
storm id 0
added polygon
1
-97.67879498032912


C:\Users\mwilson41\AppData\Local\Continuum\Anaconda3\envs\radar\lib\site-packages\numpy\core\fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\mwilson41\AppData\Local\Continuum\Anaconda3\envs\radar\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


past mask
48.2957
found a storm
[ 83.04983772   1.51059916  34.44686996  50.84738237]
Poly lon -97.67879498032912
[-97.19349926 -97.69793959 -97.45426797 -97.35324554]
[0 1 2 3]
storm id 1
added polygon
1
-97.40472233464726
past mask
48.0233
found a storm
found a big storm
plotted a big storm
made it to beginning of loop
2
-97.45310833423348
[ 48.62868708  34.81830982   0.09962385  16.32321858]
Poly lon -97.45310833423348
[-97.19349926 -97.69793959 -97.45426797 -97.35324554]
[0 1 2 3]
storm id 2
added polygon
2
-97.36212798321206
nope
2
-97.5789539153865
[ 53.28610404  30.63185974   9.24346237  22.39345425]
Poly lon -97.5789539153865
[-97.19349926 -97.69793959 -97.45426797 -97.35324554]
[0 1 2 3]
storm id 2
added polygon
2
-97.34862643068297
[ 31.46657699  51.98861946  17.29666943   0.89232765]
Poly lon -97.34862643068297
[-97.19349926 -97.69793959 -97.45426797 -97.35324554]
[0 1 2 3]
storm id 3
added polygon
made it here
made it through zdr centroids
1
-97.24290528034058
[  1.71314951

...

past mask
48.8179
found a storm
found a big storm
plotted a big storm
made it to beginning of loop
2
-97.68510128623059
[ 87.75532188   0.14219776  35.62698512  53.85414867]
Poly lon -97.68510128623059
[-98.26336411 -97.68613366 -97.45222813 -97.32939147]
[4 1 2 3]
storm id 1
added polygon
1
-97.39009350849176
past mask
48.2892
found a storm
found a big storm
plotted a big storm
made it to beginning of loop
2
-97.46597219043979
[ 73.95494935  35.88095986   1.22041999  18.19252474]
Poly lon -97.46597219043979
[-98.26336411 -97.68613366 -97.45222813 -97.32939147]
[4 1 2 3]
storm id 2
added polygon
2
-97.30982613135299
nope
1
-97.3320791755104
past mask
46.7177
found a storm
[ 74.54191315  54.39745599  18.62949341   0.6047426 ]
Poly lon -97.3320791755104
[-98.26336411 -97.68613366 -97.45222813 -97.32939147]
[4 1 2 3]
storm id 3
added polygon
made it here
made it through zdr centroids
1
-97.68955298695802
[ 85.53391602   1.37601598  34.71206613  53.25213303]
1
-97.47366640820388
[ 72.10276

C:\Users\mwilson41\AppData\Local\Continuum\Anaconda3\envs\radar\lib\site-packages\ipykernel_launcher.py:641: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


figure saved
in loop
21
in loop
21
in loop
21
in loop
21
in loop
21
in loop
21
in loop
21
0.451015
2017-07-11 23:09:43.799000
its this line
heres the problem
almost gridding
made it to smoothing
made it through gradient
got gradient
Set up our projection
(-98.825555555555553, 46.027777777777779, <cartopy.crs.PlateCarree object at 0x00000205053D9200>)
plotting
1
-98.23587680599688
past mask
46.2283
found a storm
[  0.64864618  86.48019232  72.14144206  72.54688819]
Poly lon -98.23587680599688
[-98.24461425 -97.68510129 -97.46597219 -97.33207918]
[4 1 2 3]
storm id 4
added polygon
1
-97.61599698309801
past mask
48.3339
found a storm
found a big storm
plotted a big storm
made it to beginning of loop
2
-97.67367081696723
[ 87.66019221   0.95921583  35.75871397  54.24172864]
Poly lon -97.67367081696723
[-98.24461425 -97.68510129 -97.46597219 -97.33207918]
[4 1 2 3]
storm id 1
added polygon
1
-97.52909466315916
past mask
45.0164
1
-97.37014827049656
past mask
47.7599
found a storm
found a bi

C:\Users\mwilson41\AppData\Local\Continuum\Anaconda3\envs\radar\lib\site-packages\ipykernel_launcher.py:641: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


figure saved
in loop
21
in loop
21
in loop
21
in loop
21
in loop
21
in loop
21
in loop
21


In [ ]:
#Save off the dataframe as a pickle file
tracks_dataframe.to_pickle('ExampleCase_7_11_17KMVX.pkl')